In [20]:
import requests
import pandas as pd
from bs4 import BeautifulSoup, Comment
import warnings


def scrape_player_stats(player_name, url, table_id="stats_standard", season=None):
    warnings.filterwarnings("ignore")
    headers = {'User-Agent': 'Mozilla/5.0'}
    response = requests.get(url, headers=headers, verify=False)
    soup = BeautifulSoup(response.content, 'html.parser')
    table = soup.find("table", id =table_id) if table_id else soup.find("table")
    if not table:
        for comment in soup.find_all(string=lambda text: isinstance(text, Comment)):
            if 'stats_standard' in comment:
                comment_soup = BeautifulSoup(comment, 'html.parser')
                table = comment_soup.find('table', id='stats_standard')

                thead = table.find('thead')
                theader = thead.find_all('th')
                columns = [ele.text.strip() for ele in theader]
                n = 7
                sub_columns = columns[n:]
                tbody = table.find('tbody')
                rows = tbody.find_all('tr')
                f = []
                for row in rows:
                    cells = row.find_all(['td', 'th'])
                    if len(cells) > 0:
                        data = [cell.text.strip() for cell in cells]
                        f.append(data)
                df2 = pd.DataFrame(f, columns=sub_columns)
            

                if not table:
                    print(f"Table not found for {player_name}")
                    return None

                df2 = pd.read_html(str(table), header=1)[0]
                df2['League'] = player_name
                #df2['Season'] = season

                return df2


'''
    for comment in soup.find_all(string=lambda text: isinstance(text, Comment)):
        if 'stats_standard' in comment:
            comment_soup = BeautifulSoup(comment, 'html.parser')
            table = comment_soup.find('table', id='stats_standard')
'''

"\n    for comment in soup.find_all(string=lambda text: isinstance(text, Comment)):\n        if 'stats_standard' in comment:\n            comment_soup = BeautifulSoup(comment, 'html.parser')\n            table = comment_soup.find('table', id='stats_standard')\n"

In [21]:
#https://fbref.com/en/comps/20/stats/Bundesliga-Stats
years = ["2024-2025","2023-2024","2022-2023","2021-2022","2020-2021","2019-2020","2018-2019","2017-2018","2016-2017","2015-2016"]
leagueinfo = [
    {
        "name": "Bundesliga",
        "url": "https://fbref.com/en/comps/20/{year}/stats/{year}-Bundesliga-Stats"
    },
    {
        "name": "Premier League",
        "url": "https://fbref.com/en/comps/9/{year}/stats/{year}-Premier-League-Stats"
    },
    {
        "name": "La Liga",
        "url": "https://fbref.com/en/comps/12/{year}/stats/{year}-La-Liga-Stats"
    },
    {
        "name": "Serie A",
        "url": "https://fbref.com/en/comps/11/{year}/stats/{year}-Serie-A-Stats"
    },
    {
        "name": "Ligue 1",
        "url": "https://fbref.com/en/comps/13/{year}/stats/{year}-Ligue-1-Stats"
    },
    {
        "name": "Eredivisie",
        "url": "https://fbref.com/en/comps/23/{year}/stats/{year}-Eredivisie-Stats"
    },
    {
        "name": "Primeira Liga",
        "url": "https://fbref.com/en/comps/32/{year}/stats/{year}-Primeira-Liga-Stats"
    },
    {
        "name": "Championship",
        "url": "https://fbref.com/en/comps/10/{year}/stats/{year}-EFL-Championship-Stats"
    },
    {
        "name": "Scottish Premiership",
        "url": "https://fbref.com/en/comps/40/{year}/stats/{year}-Scottish-Premiership-Stats"
    }
 
]

leagues = []
for year in years:
    for league in leagueinfo:
        leagues.append({
            "name": f"{league['name']} {year}",
            "url": league["url"].format(year=year),
            "season": year.replace("-", "/"),
        })

league_dfs = {}
combined_list = []
for league in leagues:
    df = scrape_player_stats(league["name"], league["url"])
    if df is not None:
        league_dfs[league["name"]] = df
        combined_list.append(df)

ValueError: 23 columns passed, passed data had 25 columns

In [ ]:
df = pd.concat(combined_list,ignore_index=True)

df

,Rk,Player,Nation,Pos,Squad,Age,Born,MP,Starts,Min,...,Ast,G+A,G-PK,G+A-PK,xG,xAG,xG+xAG,npxG,npxG+xAG,Matches
0,1,Rodrigo Abascal,uy URU,DF,Boavista,30,1994,29,29,"2,599",...,0.00,0.00,0.00,0.00,0.03,0.06,0.09,0.03,0.09,Matches
1,2,Nelson Abbey,eng ENG,DF,Rio Ave,20,2003,8,7,635,...,0.00,0.00,0.00,0.00,0.01,0.02,0.03,0.01,0.03,Matches
2,3,Bilal Abdelrahman Zeeni,eg EGY,MF,Estrela,21,2003,1,0,8,...,0.00,0.00,0.00,0.00,0.18,0.81,0.98,0.18,0.98,Matches
3,4,Giorgi Aburjania,ge GEO,MF,AVS Futebol,29,1995,9,3,306,...,0.00,0.00,0.00,0.00,0.09,0.17,0.26,0.09,0.26,Matches
4,5,Adriano,br BRA,MF,Santa Clara,24,1999,32,31,"2,630",...,0.07,0.07,0.00,0.07,0.04,0.09,0.13,0.04,0.13,Matches
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3479,480,Ron-Robert Zieler,de GER,GK,Stuttgart,28,1989,34,34,"3,060",...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,Matches
3480,481,Matthias Zimmermann,de GER,DF,Stuttgart,25,1992,3,1,77,...,0.00,0.00,0.00,0.00,0.06,0.00,0.06,0.06,0.06,Matches
3481,482,Simon Zoller,de GER,"MF,FW",Köln,26,1991,17,12,963,...,0.09,0.28,0.19,0.28,0.26,0.04,0.30,0.26,0.30,Matches
3482,483,Steven Zuber,ch SUI,"FW,DF",Hoffenheim,25,1991,20,17,"1,431",...,0.13,0.19,0.06,0.19,0.06,0.08,0.14,0.06,0.14,Matches


In [ ]:
league_dfs =  pd.concat(combined_list, ignore_index=True)
league_dfs['League'].unique().tolist()

['Serie A 2024-2025',
 'Ligue 1 2024-2025',
 'La Liga 2019-2020',
 'Serie A 2019-2020',
 'Primeira Liga 2019-2020',
 'Serie A 2016-2017',
 'Eredivisie 2015-2016']

In [ ]:
df = pd.concat(combined_list, ignore_index=True)
df['League'].unique().tolist()


['Serie A 2024-2025',
 'Ligue 1 2024-2025',
 'La Liga 2019-2020',
 'Serie A 2019-2020',
 'Primeira Liga 2019-2020',
 'Serie A 2016-2017',
 'Eredivisie 2015-2016']